In [1]:
import os

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"  # Arrange GPU devices starting from 0
os.environ["CUDA_VISIBLE_DEVICES"]= "1"  # Set the GPU 1 to use
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"


print("CUDA_DEVICE_ORDER:", os.environ.get("CUDA_DEVICE_ORDER"))
print("CUDA_VISIBLE_DEVICES:", os.environ.get("CUDA_VISIBLE_DEVICES"))

CUDA_DEVICE_ORDER: PCI_BUS_ID
CUDA_VISIBLE_DEVICES: 1


In [2]:
checkpoint = "jiwon65/whisper-small_korean-zeroth"

# Whisper model output은 vocabulary 단어의 index
# 이를 실제 문자와 mapping하기 위해 Tokenizer 사용

from transformers import WhisperTokenizer
from transformers import WhisperProcessor
from transformers import WhisperFeatureExtractor

tokenizer = WhisperTokenizer.from_pretrained(checkpoint, language="Korean", task="transcribe")
feature_extractor = WhisperFeatureExtractor.from_pretrained(checkpoint)
processor = WhisperProcessor.from_pretrained(checkpoint, language="Korean", task="transcribe")

tokenizer_config.json:   0%|          | 0.00/805 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.08k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


preprocessor_config.json:   0%|          | 0.00/339 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [6]:
from transformers import WhisperForConditionalGeneration

model_tuned = WhisperForConditionalGeneration.from_pretrained("../checkpoint-350/")
model_base = WhisperForConditionalGeneration.from_pretrained("jiwon65/whisper-small_korean-zeroth")

config.json:   0%|          | 0.00/1.32k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/967M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.83k [00:00<?, ?B/s]

In [7]:
import torch
print("Number of GPUs available:", torch.cuda.device_count())

device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")

Number of GPUs available: 1


In [8]:
import torch
import torchaudio
from transformers import WhisperProcessor, WhisperForConditionalGeneration

class WhisperTranscriber:
    def __init__(self, processor, model, device):
        self.processor = processor
        self.model = model
        self.device = device
        self.model.to(self.device)

    def transcribe(self, wav_file_path):
        # Load the audio file
        audio, sample_rate = torchaudio.load(wav_file_path)
        
        # Resample the audio to 16kHz if needed
        if sample_rate != 16000:
            resampler = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=16000)
            audio = resampler(audio)

        # Preprocess the audio to get input features
        input_features = self.processor(audio.squeeze().numpy(), sampling_rate=16000, return_tensors="pt").input_features

        # Generate the transcription
        with torch.no_grad():
            predicted_ids = self.model.generate(input_features.to(self.device))

        # Decode the predicted ids to text
        transcription = self.processor.batch_decode(predicted_ids, skip_special_tokens=True)[0]
        return transcription

def compute_cer(labels, transcription):
    pred = []
    lab = []
    N = 0
    corr = 0
    for tr in transcription:
        if (tr != " "):
            pred.append(tr)
    
    for tr in labels:
        if (tr != " "):
            N +=1
        if (tr in pred):
            corr +=1
    cer =  (1 - corr/N)*100

    return cer

In [9]:
from IPython.display import Audio, display

import glob, os
from datasets import load_dataset
file_path = '/home/nas3/project/2023_LG_HA/data/240911_AEC_simulation_dataset/processed_output/'
comparing_raw_data = sorted(glob.glob(os.path.join(file_path, "노인남여_노인대화09_F_1533066829_61_수도권_실내/*.wav"),recursive=True))
MAX_LEN_RAW = len(comparing_raw_data)
MAX_LEN_RAW

0

In [62]:
import soundfile as sf
import librosa
file_path = '/home/nas3/project/2023_LG_HA/data/240911_AEC_simulation_dataset/processed_output/'


transcriber = WhisperTranscriber(processor, model_tuned, device)
idx = 6
files = glob.glob(file_path + '*.wav')
temp, _ = librosa.load(files[idx], sr=16000)
transcription = transcriber.transcribe(files[idx])

In [63]:
print(transcription)

스마트 터보 모두 설명해 줘


In [64]:
Audio(temp, rate=16000)


In [26]:
temp, _ = librosa.load(files[0], sr=16000)
